In [48]:
import nltk
import re
import numpy as np
from nltk.corpus import gutenberg
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [49]:
def generate_seq(model, tokenizer, seed_text, n_words):
    in_text, result = seed_text, seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        print(encoded)
        encoded = array(encoded)
        print(encoded)
        # predict a word in the vocabulary
        yhat = model.predict_classes(encoded, verbose=0)
        print(yhat)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text, result = out_word, result + ' ' + out_word
    return result

In [50]:
def get_source_text():
    #for key in gutenberg.fileids():
    sentence_vector = gutenberg.sents('austen-emma.txt')
    #length[key] =  len(sentence_vector)
    length = len(sentence_vector)
    global_text = []
    for sentence in sentence_vector[:int(length * 0.9)]:
        text = " <s> " + ' '.join(re.compile(r"\w+").findall(' '.join(sentence))).lower() +" </s> "
        global_text.append(text)
    return global_text

In [47]:
data = get_source_text()
print(data)

[u' <s> emma by jane austen 1816 </s> ', u' <s> volume i </s> ', u' <s> chapter i </s> ', u' <s> emma woodhouse handsome clever and rich with a comfortable home and happy disposition seemed to unite some of the best blessings of existence and had lived nearly twenty one years in the world with very little to distress or vex her </s> ', u' <s> she was the youngest of the two daughters of a most affectionate indulgent father and had in consequence of her sister s marriage been mistress of his house from a very early period </s> ', u' <s> her mother had died too long ago for her to have more than an indistinct remembrance of her caresses and her place had been supplied by an excellent woman as governess who had fallen little short of a mother in affection </s> ', u' <s> sixteen years had miss taylor been in mr woodhouse s family less as a governess than a friend very fond of both daughters but particularly of emma </s> ', u' <s> between _them_ it was more the intimacy of sisters </s> ', u

In [51]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [52]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 6703


In [53]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
    sequence = encoded[i-1:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 6975


In [54]:
# split into X and y elements
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

In [55]:
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)

In [56]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1, 10)             67030     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_4 (Dense)              (None, 6703)              677003    
Total params: 788,433
Trainable params: 788,433
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=2, verbose=2)

Epoch 1/2
 - 39s - loss: 8.8443 - acc: 0.0136
Epoch 2/2
 - 12s - loss: 8.8081 - acc: 0.0165


In [43]:
print(generate_seq(model, tokenizer, 'my', 9))

[]
[]


AttributeError: 'list' object has no attribute 'shape'

SyntaxError: invalid syntax (<ipython-input-26-87f21140cc1a>, line 3)